In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# Workbook

| Date       | Version   | Description                                      | Train score | Public score  | Commit Version
|--------------------------------------------------------------------------------------------------------------------------------------------|
| 2019-06-01 | v1.0        | resnet50 transfer learning, loss='categorical_crossentropy'  | - | -  | v6 |
| 2019-06-07 | v1.1        | resnet50 transfer learning,  kappa scoree  |  - | -| v7 |


In [ ]:
# Load the extension and start TensorBoard

%load_ext tensorboard.notebook
%tensorboard --logdir logs

In [ ]:
train_df = pd.read_csv("../input/petfinder-adoption-prediction/train/train.csv")
train_data_dir = '../input/petfinder-train-keras/petfinder_train_keras/train_img'

In [3]:
!pip uninstall wrapt
!pip install wrapt==1.11.1

Cannot uninstall 'wrapt'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
  Found existing installation: wrapt 1.10.11
Cannot uninstall 'wrapt'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
#import keras
#import tensorflow as tf
#from keras import backend as K

#print(keras.__version__)
#print(tf.__version__)

!pip install --ignore-installed tensorflow==2.0.0-beta0

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

print(tf.VERSION)

    100% |████████████████████████████████| 87.9MB 122kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/bb/10/44230dd6bf3563b8f227dbf344c908d412ad2ff48066476672f3a72e174e/wheel-0.33.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/28/6a/8c1f62c37212d9fc441a7e26736df51ce6f0e38455816445471f10da4f0a/Keras_Preprocessing-1.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d1/4f/950dfae467b384fc96bc6469de25d832534f6b4441033c39f914efd13418/astor-0.8.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a4/96/571b875cd81dda9d5dfa1422a4f9d749e67c0a8d4f4f0b33a4e5f5f35e27/tb_nightly-1.14.0a20190603-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/32/dd/99c47dd007dcf10d63fd895611b063732646f23059c618a373e85019eb0e/tf_estimator_nightly-1.14.0.dev2019060501-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d2/fb/29de8d08967f0cce1bb10b39846d836b0f3bf

In [ ]:
def _cohen_kappa(y_true, y_pred, num_classes, weights=None, metrics_collections=None, updates_collections=None, name=None):
   kappa, update_op = tf.contrib.metrics.cohen_kappa(y_true, y_pred,num_classes, weights, metrics_collections, updates_collections, name)
   #K.get_session().run(tf.local_variables_initializer())
   #with tf.control_dependencies([update_op]):
      #kappa = tf.identity(kappa)
   #return kappa
   return update_op

def cohen_kappa_loss(num_classes, weights=None, metrics_collections=None, updates_collections=None, name=None):
   def cohen_kappa(y_true, y_pred):
      return -_cohen_kappa(y_true, y_pred, num_classes, weights, metrics_collections, updates_collections, name)
   return cohen_kappa

In [ ]:
def cohens_kappa(y_true, y_pred):
    y_true_classes = tf.argmax(y_true, 1)
    y_pred_classes = tf.argmax(y_pred, 1)
    return tf.contrib.metrics.cohen_kappa(y_true_classes, y_pred_classes, 5)[1]

In [ ]:
def get_weighted_loss():
    def weighted_loss(y_true, y_pred):
        return K.mean( K.square(y_pred - y_true) * K.exp(-K.log(1.7) * (K.log(1. + K.exp((y_true - 3)/5 )))),axis=-1  )
    return weighted_loss

In [ ]:
def my_kappa_loss(num_classes):
    def my_kappa(y_true, y_pred):
        #y_true_classes = tf.argmax(y_true, 1)
        #y_pred_classes = tf.argmax(y_pred, 1)
        y_true = K.flatten(y_true)          
        y_pred = K.flatten(y_pred)
        #y_true = K.cast(y_true, tf.float64)  
        #y_pred = K.cast(y_pred, tf.float64)  
        return -tf.contrib.metrics.cohen_kappa(y_true, y_pred, num_classes)[1]
    return my_kappa

In [ ]:
def cohens_kappa(y_true, y_pred):
    y_true_classes = tf.argmax(y_true, 1)
    y_pred_classes = tf.argmax(y_pred, 1)
    return tf.contrib.metrics.cohen_kappa(y_true_classes, y_pred_classes, 5)[1]

In [ ]:
#from tf.keras.applications import ResNet50
#from tf.keras.models import Sequential
#from tf.keras.layers import Dense, Flatten, GlobalAveragePooling2D

num_classes = 5
model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, pooling='avg')

my_new_model = tf.keras.models.Sequential()
my_new_model.add(model)
my_new_model.add(layers.Dense(num_classes, activation='softmax'))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

# get the loss function and set parameters
model_cohen_kappa = my_kappa_loss(num_classes=num_classes)

#sgd Stochastic gradient descent optimizer
#adam other
my_new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[cohens_kappa])

print(my_new_model.summary())

In [ ]:
from tf.keras.utils import plot_model
plot_model(my_new_model)

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard


image_size = 224
batch_size = 64
validation_split = 0.1
nb_epochs = 20

# steps_per_epoch: number of yields (batches) before a epoch is over
# ceil(num_samples / batch_size)
# epochs: Number of epochs to train the model. An epoch is an iteration over the entire data provided
# class_weight: Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only). 
#  This can be useful to tell the model to "pay more attention" to samples from an under-represented class.

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=validation_split) # set validation split

#liefert die trainingsdaten als iterator
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

#liefert die Validationdaten als iterator
validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

STEP_SIZE_TRAIN=math.ceil(train_generator.n//train_generator.batch_size)
STEP_SIZE_VALID=math.ceil(validation_generator.n//validation_generator.batch_size)

print(STEP_SIZE_TRAIN)
print(STEP_SIZE_VALID)


# Configure the TensorBoard callback and fit the model
tensorboard_callback = TensorBoard("logs")

#K.get_session().run(tf.local_variables_initializer())
#K.get_session().run(tf.global_variables_initializer())
sess = tf.Session()
sess.run(tf.global_variables_initializer())
#sess.run(tf.local_variables_initializer())

history = my_new_model.fit_generator(
    train_generator,
    steps_per_epoch = STEP_SIZE_TRAIN,
    validation_data = validation_generator, 
    validation_steps = STEP_SIZE_VALID,
    epochs = nb_epochs,
    callbacks=[tensorboard_callback])

In [ ]:
import matplotlib.pyplot as plt
def plot_training(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    plt.savefig('acc_vs_epochs.png')
    
plot_training(history)